In [7]:
import random
from collections import defaultdict

In [8]:
aggregators = ["counter", "gauge", "sketch", "exact", "histogram"]
records = []

In [9]:
f = open("PrometheusDataSecond.csv", "w")

for i in range(2048):
    agg = random.choice(aggregators)

    nam = "name" + str(i) + "_" + agg
    des = "description" + str(i)
    lab = "{key" + str(i) + ":value" + str(i) + "}"
    ndl = str(nam) + ", " + str(des) + ", " + str(lab)
    ndl = ndl.replace("[","").replace("]","").replace("'","")
#     print(ndl)


    if (agg == "counter" or agg == "gauge"):
        # Last value
        val = random.randint(-50,50)
        
        record = (agg, ndl, val)
        records.append(record)
        
        f.write(agg+'|'+str(val)+'|'+ndl+'\n')
#         print(agg+'|'+str(val)+'|'+ndl+'\n')


    #TODO: test exact in order mode?
    elif (agg == "sketch" or agg == "exact"):
        # Array of update values - extract sum/count from this array
        count = random.randint(1, 5)
        update_values = []
        for i in range(count):
            update_values.append(random.randint(-50, 50))
        update_values = list(set(update_values))
        
        # Array of quantiles
        quantiles = []
        
        # Randomly generated "user specified" quantiles for sketch
        if agg == "sketch":
            for _ in range(5):
                quantiles.append(random.randint(0, 100)/100)
                quantiles.sort()
            quantiles = list(set(quantiles))

        # Pre defined quantiles for exact
        elif agg == "exact":
            quantiles = [0, 0.5, 0.9, 0.95, 0.99, 1]

        record = (agg, ndl, update_values, quantiles)
        records.append(record)
        
        update_val_str = "[" + str(update_values[0])
        for v in update_values[1:]:
            update_val_str = update_val_str + "," + str(v)
        update_val_str += "]"
        
        quantiles_str = "[" + str(quantiles[0])
        for v in quantiles[1:]:
            quantiles_str = quantiles_str + "," + str(v)
        quantiles_str += "]"
        
        f.write(agg+'|'+update_val_str + '|' + quantiles_str+'|'+ndl+'\n')
#         print(agg+'|'+update_val_str + '|' + quantiles_str+'|'+ndl+'\n')
        
    elif (agg == "histogram"):
        # Array of update values - extract sum/count from this array
        count = random.randint(1, 5)
        update_values = []
        for i in range(count):
            update_values.append(random.randint(1, 100))
        update_values = list(set(update_values))
        
        # Array of boundaries
        boundaries = []
        for _ in range(5):
            boundaries.append(random.randint(0, 100))
            boundaries.sort()
        boundaries = list(set(boundaries))
        
        record = (agg, ndl, update_values, boundaries)
        records.append(record)
        
        update_val_str = "[" + str(update_values[0])
        for v in update_values[1:]:
            update_val_str = update_val_str + "," + str(v)
        update_val_str += "]"
        
        boundaries_str = "[" + str(boundaries[0])
        for v in boundaries[1:]:
            boundaries_str = boundaries_str + "," + str(v)
        boundaries_str += "]"

        f.write(agg+'|'+update_val_str + '|' + boundaries_str+'|'+ndl+'\n')
#         print(agg+'|'+update_val_str + '|' + boundaries_str+'|'+ndl+'\n')
    
    
f.close()

In [10]:
import math

fa = open("PrometheusAnswersSecond.csv", "w")

for record in records:
    agg = record[0]    
    
    # record = (agg, ndl, val)
    if (agg == "counter" or agg == "gauge"):
        fa.write(agg + '|' + str(record[2]) + '|' + record[1] + '\n')
#         print("{}|{}|{}".format(agg, str(record[2]), record[1]))
        
    
    # record = (agg, ndl, update_values, quantiles)
    elif (agg == "sketch" or agg == "exact"):
        s = sum(record[2])
        c = len(record[2])
        quantile_values = []
        
        for quantile in record[3]:
            pos = (len(record[2])-1) * quantile
            ceiling = math.ceil(pos)
            quantile_values.append(record[2][ceiling])
        
        quantile_pairs = "[(" + str(record[3][0]) + "," + str(quantile_values[0]) + ")"
        for q, v in zip(record[3][1:], quantile_values[1:]):
            quantile_pairs = quantile_pairs + ",(" + str(q) + "," + str(v) + ")"
        quantile_pairs += "]"
        
        fa.write("summary|" + str(s) + "|" + str(c) + "|" + quantile_pairs + "|" + record[1] + '\n')
#         print("summary|{}|{}|{}|{}".format(s, c, quantile_pairs, record[1]))


    #record = (agg, ndl, update_values, boundaries)
    elif (agg == "histogram"):
        s = sum(record[2])
        c = len(record[2])
        
        
        # buckets is a dict of that correspond to pairs of (buckets, bucket_count)
        buckets = defaultdict(int)
        
        update_values_sorted = sorted(record[2])
        curr_boundary = -1
        values_within_bound = 0
        for boundary in record[3]:
            if(values_within_bound != len(update_values_sorted)):
                for v in update_values_sorted:
                    if v < boundary and v > curr_boundary:
                        buckets[str(boundary)] += 1
                        values_within_bound += 1
                    elif v < boundary and v <= curr_boundary:
                        continue
                    else:
                        curr_boundary = boundary
                        break
        if(len(update_values_sorted) - values_within_bound != 0):
            buckets["inf"] += len(update_values_sorted) - values_within_bound

        buckets_str = "["
        for k, v in buckets.items():
            buckets_str = buckets_str + "(" + k + "," + str(v) + "),"
        buckets_str += "]"
        
        # extra comma will be present at the end of bucket_str; no functional impact
        fa.write("histogram|" + str(s) + "|" + str(c) + "|" + buckets_str + "|" + record[1] + '\n')
#         print("histogram|{}|{}|{}|{}".format(s, c, buckets_str, record[1]))
            
fa.close()